# Computing integer square roots

**Definition.** Given a nonnegative integer $n$, the greatest integer $a$ such that $a^2\le n$ is the **integer square root** of $n$.

Less formally, the integer square root of $n$ is simply the integer part of the square root of $n$. For example, if $n = 23$, the square root of $n$ is

$$\sqrt{23} = 4.7958315233127195415974380641626939199967070419041293...$$
    
so the integer square root of $23$ is $4$.

This notebook presents and compares various methods for computing the integer square root in Python. In particular, we present a simple (and apparently new) adaptive-precision pure-integer-based method based on Newton-Raphson.

## Floating-point-based algorithms

The following implementation seems obvious, but has some problems. Python's `math` module has a `sqrt` function which gives a floating-point approximation to the square root of its input. So we can simply call `sqrt` and then extract and return the integer part.

In [1]:
import math

def isqrt(n):
    """Integer square root of a positive integer; float-based algorithm."""
    return int(math.sqrt(n))

This is fast and simple, and seems to work well for small integers:

In [2]:
isqrt(11)

3

In [3]:
isqrt(25)

5

In [4]:
isqrt(99)

9

However, this implementation of `isqrt` will fail for larger values. For example, the following will give the wrong result on a typical machine.

In [5]:
n = 2**52 + 2**27
a = isqrt(n)  # one too large
a*a <= n

False

The problem here is that the exact square root is just slightly smaller than `2**26 + 1`; the difference between the exact square root and `2**26 + 1` is a touch smaller than `2**-27`. Since the exact square root is not exactly representable as a float, it gets rounded to the nearest value that *is* exactly representable as a float, and in this case that value happens to be `2**26 + 1`.

The exact threshold at which things start to go wrong will depend on the floating-point format in use together with the accuracy of `math.sqrt`, but assuming IEEE 754 binary64 floating-point, round-ties-to-even rounding mode, and a correctly rounded `math.sqrt`, the above `n` is the smallest for which the wrong result will be returned.

Sketch of proof: suppose that we're working with a binary floating-point format with precision $p$, and that $4^{k - 1} \le n < 4^k$ with $2k \le p$. Write $a$ for the integer square root of $n$, so $2^{k-1} \le a < 2^k$. Then from the bound on $k$,

$$ a < 2^{p-k}\tag1$$ 

from which it follows easily that

$$ a + 1 + \sqrt{a^2 + 2a} < 2^{p-k+1}\tag4$$

Inverting and rearranging gives

$$ \sqrt{a^2 + 2a} < a + 1 - 2^{k-p-1}\tag5$$

Hence

$$ a \le \sqrt{n} \le \sqrt{a^2 + 2a} < a + 1 - 2^{k-p-1}\tag6$$

The spacing between successive floats in the interval $[2^{k-1}, 2^k]$ is $2^{k-p}$. So under round-ties-to-even, $a + 1 - 2^{k-p-1}$ is the least value which will round up to $a+1$, so $\sqrt{n}$ must round to a value smaller than $a+1$. Since $a$ is exactly representable, $\sqrt{n}$ can't round to something smaller than $a$. So the integer part of the rounded $\sqrt{n}$ will be exactly $a$.

For the particular case of the IEEE 754 binary64 format, we have $p = 53$, and so the above tells us that `int(sqrt(n))` will give the correct result for any $n < 2^{52}$.

For the rest, it's easy to verify that if $n = 2^{52} + 2^{27} - 1$ then `int(sqrt(n))` is $2^{26}$, and so this must also be true for all values between $2^{52}$ and $2^{52} + 2^{27} - 1$. It's also easy to verify that for $n = 2^{52} + 2^{27}$, $\sqrt{n}$ rounds up to $2^{26} + 1$.


The following also fails, because $2^{1024}$ exceeds the range of an IEEE 754 binary64 float.

In [6]:
n = 2**1024
try:
    isqrt(n)
except Exception as e:
    print(repr(e))

OverflowError('int too large to convert to float')


There's a more subtle problem: if `math.sqrt` isn't perfectly correctly rounded, the `isqrt` implementation risks giving bad answers even for small inputs. For example, if `math.sqrt(9)` happens to give `2.9999999999999996` instead of exactly `3.0`, then `isqrt(9)` will give `2` instead of `3`. The chances are extremely good that your implementation of `math.sqrt` _is_ correctly rounded, but Python provides no guarantees. If you're worried about this, a safer option is:

In [7]:
def isqrt(n):
    """ Integer square root via floating-point.
    
    More tolerant of numeric errors than the simple version.
    """
    return int(math.sqrt(n + 0.5))

In [8]:
isqrt(9)

3

But this still fails for larger values, just like our original function does. If we want a general solution, we need to work harder.

**Analysis.** Suppose again that we're working with precision-$p$ binary floating-point, and that $4^{k-1} \le n < 4^{k}$ for some $k$. We also assume that $n < 2^{p-1}$, so that $n + 0.5$ can be represented exactly as a floating-point number. Write $a$ for the integer square root of $n$, so $2^{k-1} \le a < 2^{k}$, $a^2 \le n \le a^2 + 2a$, and the spacing between successive floats in $[2^{k-1}, 2^k]$ is $2^{k-p}$.

Then

$$
2^{k-1}\le a < \sqrt{a^2 + \frac12}
             \le \sqrt{n + \frac12}
             \le \sqrt{a^2 + 2a + \frac12}
             < a + 1 \le 2^k.
$$

and it's easy to show that

$$ \sqrt{a^2 + \frac12} - a > 2^{-k-2} $$
and
$$ a + 1 - \sqrt{a^2 + 2a + \frac12} > 2^{-k-2}.$$

It follows that if the error in computing $\sqrt{n + \frac12}$ using `math.sqrt` is bounded by $2^{-k-2}$, then the above method will give the correct result. So it's enough that `math.sqrt` is accurate to within $2^{p - 2k - 2}$ units in the last place.

Turning this around, if `math.sqrt` is uniformly accurate to within $2^e$ units in the last place, then the above method will give accurate results for $n$ up to $4^{\left\lfloor(p - e - 2) / 2\right\rfloor}$. In the special case where $e = -1$ (correct rounding) and $p = 53$ (IEEE 754 binary64 format), this shows that the method above continues to give correct results for all $n < 2^{52}$, so is comparable to the previous method.

Assuming a correctly rounded `math.sqrt` and IEEE 754 binary64 floating-point, the smallest $n$ for which things fail is $n = 2^{52} + 2^{27} - 1$, one smaller than for the previous algorithm. In this case, $n + 0.5$ is already not representable as a float, so the result of `n + 0.5` is actually equal to `n + 1`.

## Version 2: bisection search

We can avoid floating-point entirely and use pure integer arithmetic. The following code uses a simple bisection search to identify the integer square root. To get an initial upper bound, it makes use of the fact that for any positive integer `n`, `n < 2**n.bit_length()`.

In [9]:
def isqrt(n):
    """ Integer square root via bisection search. """
    lower = 0
    upper = 1 << -(-n.bit_length() // 2)
    while upper - lower > 1:
        midpoint = (lower + upper) // 2
        if midpoint * midpoint <= n:
            lower = midpoint
        else:
            upper = midpoint
    return lower    

Note: to keep the code simple, we're ignoring the case where `n` is negative. We'll do this throughout this notebook.

To make sure we haven't messed up, let's run some tests.

In [10]:
def is_isqrt(n, a):
    """
    True if a is the integer square root of n.
    """
    return a * a <= n < (a + 1) * (a + 1)

def test_isqrt():
    assert all(is_isqrt(n, isqrt(n)) for n in range(10**6))
    assert isqrt(2**1024 - 1) == 2**512 - 1
    assert isqrt(2**1024 + 1) == 2**512    

In [11]:
test_isqrt()

This algorithm is simple, and its correctness is easy to verify. But it's a bit on the slow side when ``n`` gets large. Let's set up some timings. We'll use 1000 different randomly-chosen 1000-digit integers for profiling.

In [12]:
import random
random.seed(56176)
profile_values = [random.randrange(10**1000) for _ in range(1000)]

def profile_isqrt():
    return [isqrt(n) for n in profile_values]

In [13]:
%timeit profile_isqrt()

3.98 s ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Can we do better? Yes. We. Can.

## Version 3: Newton's method

The one-dimensional version of the Newton-Raphson method says that if $f$ is a real-valued function of a real input, and $x_0$ is sufficiently close to a root of $f$, then under some well-behavedness assumptions on $f$ (see below),

$$x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}$$

will be a better approximation to the root. Iterating then gives quadratic convergence to the root: the number of valid decimal digits approximately doubles at each step.

If $n$ is a positive real number and $f$ is the function given by $f(x) = x^2 - n$, then a root of $f$ is a square root of $n$, convergence is guaranteed from any nonzero starting value, and the formula becomes:

$$x_1 = \frac12 \left(x_0 + \frac n{x_0}\right)$$

**Small print.** About those well-behavedness assumptions: it's enough for example that $f$ is twice continuously differentiable in a neighbourhood of the root, and that $f'$ doesn't vanish at the root.

A variant of this gives an efficient pure-integer algorithm for integer square roots, though getting the termination condition right is delicate.

In [14]:
def isqrt(n):
    """
    Integer square root, via Newton's algorithm.
    """
    # Need to special-case zero.
    if n == 0:
        return 0
    
    # The initial guess must not be less than isqrt(n).
    a = 1 << -(-n.bit_length() // 2)
    while True:
        d = n // a
        if d >= a:
            return a
        a = (a + d) // 2

Let's run our tests, and profile as before.

In [15]:
test_isqrt()
%timeit profile_isqrt()

54.9 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Newton's method roughly doubles the number of accurate bits at each step. So if `isqrt(n)` has `k` bits, the last step might double accuracy from `k//2` bits to `k` bits, while the first few steps double us from `1` bit to `2` bits, or `2` bits to `4` bits of accuracy. But each iteration is being performed at full precision, with a `2*k`-bit by `k`-bit division. That's wasteful during the initial steps.

Instead of this, we can gradually increase the working precision as we go, reaching full precision only in the final step. This leads to the following recursive algorithm, still based on Newton's method.

In [16]:
def isqrt_aux(n, c, s):
    """
    Approximate integer square root of a positive integer.
    
    On input, c must be floor(log4(n >> s)); that is, one less than
    the number of digits in the base-4 expansion of n >> s.

    The return value a satisfies (a - 1)**2 < n >> s < (a + 1)**2. In
    particular, if n is a perfect square and s == 0, the returned value is
    the exact square root of n.
    """
    if not c:
        return 1
    k = (c - 1) // 2
    a = isqrt_aux(n, c // 2, s + 2*k + 2)
    return (a << k) + (n >> s+k+2) // a


def isqrt(n):
    """
    Integer square root of a nonnegative integer.
    """
    if n == 0:
        return 0
    a = isqrt_aux(n, (n.bit_length() - 1) // 2, 0)
    return a - (a*a > n)

In [17]:
test_isqrt()
%timeit profile_isqrt()

11.1 ms ± 72.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


For a slight extra speed boost, if you trust that `math.sqrt` on your system _is_ correctly rounded, and you know that the system uses the standard IEEE 754 binary64 format for its `float`s, then you can use a floating-point approximation once `n` gets small enough, and so terminate the recursion earlier. It's possible to prove that with these assumptions, if `0 < n < 2**106`, then `int(math.sqrt(n))` differs from the true square root of `n` by less than `1.0`. Here's the new `isqrt_aux` definition in that case; we keep the same `isqrt` as before.

In [18]:
def isqrt_aux(n, c):
    """
    Approximate integer square root of a positive integer.
    
    On input, c must be floor(log4(n)); that is, one less than
    the number of digits in the base-4 expansion of c

    The return value a satisfies (a - 1)**2 < n < (a + 1)**2. In
    particular, if n is a perfect square, the returned value is
    the exact square root of n.
    """
    if c < 53:
        return int(math.sqrt(n))
    k = (c - 1) // 2
    a = isqrt_aux(n >> 2*k + 2, c // 2)
    return (a << k) + (n >> k+2) // a

def isqrt(n):
    """
    Integer square root of a nonnegative integer.
    """
    if n == 0:
        return 0
    a = isqrt_aux(n, (n.bit_length() - 1) // 2)
    return a - (a*a > n)

In [19]:
test_isqrt()
%timeit profile_isqrt()

9.66 ms ± 363 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Here's an iterative version of the above. It scans the bits of `c` from left to right, and performs one big-integer division, one big-integer addition and two big-integer shifts per iteration, plus a final big-integer multiplication and comparison. All other operations are with small integers.

In [20]:
import operator

def isqrt(n):
    """
    Return the largest integer not exceeding the square root of the input.
    """
    if n == 0:
        return 0

    c = (n.bit_length() - 1) // 2
    d = 0
    a = 1
    for s in reversed(range(c.bit_length())):
        e = d
        d = c >> s
        a = (a << d - e - 1) + (n >> 2*c - e - d + 1) // a
        # Loop invariant: (a-1)**2 < n >> 2*(c - d) < (a+1)**2
        
    return a - (a*a > n)

In [21]:
test_isqrt()
%timeit profile_isqrt()

10.7 ms ± 50.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


If we're going to code this up for CPython, we'll likely want a fast path for small integers. Suppose that $2^{62} \le n < 2^{64}$. Then in the above iterative version, `c` starts at `31`, and unwinding the loop gives the following code.

In [22]:
def isqrt_small(n):
    assert 0 < n < 2**64
    # Shift left to make sure we're in range 2**62 <= n < 2**64
    c = (n.bit_length() - 1) // 2
    
    n <<= 62 - 2*c

    a = 1 + (n >> 62)  # 2 <= a <= 4, a**2 ≈ n >> 60
    a = (a << 1) + (n >> 59) // a  # 8 <= a <= 16, a**2 ≈ n >> 56
    a = (a << 3) + (n >> 53) // a  # 128 <= a <= 256, a**2 ≈ n >> 48
    a = (a << 7) + (n >> 41) // a  # 2^15 <= a <= 2^16, a**2 ≈ n >> 32
    a = (a << 15) + (n >> 17) // a  # 2^31 <= a <= 2^32, a**2 ≈ n

    a -= a * a - 1 >= n
    return a >> 31 - c;


In [23]:
test_values = [
    random.randrange(4**k, 4**(k+1))
    for k in range(32)
    for _ in range(10**4)
]
for n in test_values:
    assert isqrt_small(n) == isqrt(n)

There's a potentially annoying corner case in the above that will affect attempts to translate this to C: at the last step, `a` could be `2**32` (but no bigger), in which case `a * a` will overflow in 64-bit integer arithmetic, leading to the comparison of `a * a` with `n` to give the wrong result. But it should be enough to compare `a * a - 1` with `n` instead (using `>=` instead of `>`).


### Informal proof of correctness

It's far from obvious that this recursive square root algorithm is valid. Some of the details are straightforward, like verifying the base case; here we concentrate on the hard part, which is verifying that the desired accuracy is preserved in the recursive step. For convenience, we rewrite the Python expressions in mathematical notation.

In the recursive step, we have a positive integer $n$, a positive integer $c = \left\lfloor \log_4(n) \right\rfloor$, and a value $k = \left\lfloor \frac{c-1}2\right\rfloor$. We call `isqrt_aux(n >> 2*k + 2, c//2)` to get an approximation $a$ to the square root of $\left\lfloor\frac n {4^{k+1}}\right\rfloor$. We can assume that the output of the recursive call is accurate, satisfying:

$$(a-1)^2 < \left\lfloor \frac n{4^{k+1}}\right\rfloor < (a + 1)^2 \tag1$$

and we return a value $d$ defined by

$$d = 2^k a + \left\lfloor\frac n{2^{k+2}a}\right\rfloor. \tag2$$

We need to show that $d$ is within $1$ of the square root of $n$; i.e., that $(d - 1)^2 < n < (d + 1)^2$.

Since $(a+1)^2$ is an integer, we can remove the floor brackets in (1) to get:

$$(a-1)^2 < \frac n{4^{k+1}} < (a + 1)^2. \tag3$$

Taking square roots in (3), noting that $a$ cannot be zero, so we must have $a \ge 1$, and rearranging gives

$$ \left|\, 2^{k+1}a - \sqrt n\, \right| < 2^{k+1}. \tag4$$


Now squaring both sides and dividing by $2^{k+2} a$ gives

$$0 \le 2^k a + \frac n {2^{k+2}a} - \sqrt n < \frac{2^k}a. \tag5$$

To proceed we need a lower bound on $a$. From the definitions of $c$ and $k$ we have $4^c \le n$ and $2k \le c - 1$, so

$$ 4^{2k + 1} \le n. \tag6$$


Dividing by $4^{k+1}$, taking floors, and combining with the right-hand side of (1),

$$4^k \le \left\lfloor \frac n{4^{k+1}} \right\rfloor < (a + 1)^2. \tag7$$

Taking the square root of both sides of (7) gives $2^k < a + 1$, and since both $2^k$ and $a + 1$ are integers this implies

$$2^k \le a. \tag8$$

Now combining (5) with (8) gives

$$ 0 \le 2^k a + \frac n {2^{k+2}a} - \sqrt n < 1 \tag9$$

and taking the floor gives

$$ -1 < 2^k a + \left\lfloor\frac n{2^{k+2}a}\right\rfloor - \sqrt n < 1 \tag{10}$$

or in terms of $d$,

$$ -1 < d - \sqrt n < 1. \tag{11}$$


That says that $d$ is within $1$ of the exact square root of $n$, which is exactly what we needed to prove.